In [1]:
from app.containers import Container
container = Container()
container.config.from_yaml("config.yaml")
retriever_handler = container.retriever()

/home/huy/miniconda3/envs/llm/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


./vectorstore


In [ ]:
llm_handler = container.llm()
await llm_handler.generate_response([{"role":"user", "content":"Hello"}])

In [3]:
#retriever_handler.add_documents("./ASEAN.docx", "asian")

In [3]:
main_agent = container.main_agent()

In [ ]:
messages = [{"role":"user", "content":"How many times has Vietnam been the ASEAN chair?"}]
async for chunk in await main_agent.response(messages):
    print(chunk, end='', flush=True)

In [ ]:
default_file_metadata_func("./ASEAN.docx")

In [ ]:
from app.containers import Container
container = Container()
container.config.from_yaml("config.yaml")

container.wire(modules=[__name__])
chat_service = container.chat_service()
from uuid import UUID, uuid4
from app.schema import Message

async for content in chat_service.stream_response(id=uuid4(), message=Message(id = str(uuid4()), role="user", content="Xin chào?")):
    print(content, end='', flush=True)


In [ ]:
from uuid import UUID, uuid4
from app.schema import Message

async for content in chat_service.stream_response(id=uuid4(), message=Message(id = str(uuid4()), role="user", content="Việt Nam gia nhập ASEAN vào năm nào?")):
    print(content, end='', flush=True)


In [ ]:
#write code to load config from config.yaml
from yaml import load, Loader
with open("config.yaml", "r") as f:
    config = load(f, Loader=Loader)
config


In [8]:
import asyncio
from openai import AsyncOpenAI
client = AsyncOpenAI(api_key=config["llm"]["api_key"])
import pyaudio

async def stream_audio():
    p = pyaudio.PyAudio()
    stream = p.open(format=8,
                    channels=1,
                    rate=24_000,
                    output=True)

    async with client.audio.speech.with_streaming_response.create(
            model="tts-1",
            voice="alloy",
            input="""InvokerAI có thể nói là sự kết hợp của Automatic1111 và ComfyUI kèm với đó là sự nâng cấp đang kể về giao diện hiện đại hơn. Hiện tại InvokerAI có thể hoạt động trên cả 2 loại generate ảnh là workflow note kéo thả và generate ảnh mặc định theo giao diện có sẵn. Invoker khá chú trọng tới trải nghiệm người dùng khi cho phép quản lý ảnh và thư viện tài nguyên một cách nhất quán ngay trên giao diện sử dụng. Tuy nhiên Invoker lại không cho phép phát triển các extension mở, các update công nghệ mới khá chậm dẫn đến việc khá kén người dùng phổ thông. Ngoài ra khi sử dụng Invoker AI, các cú pháp về prompt cũng được viết khác hoàn toàn cho với ComfyUI và Automatic cũng là một điểm trừ khiến cho người dùng khó tiếp cận.""",
            response_format="pcm"
    ) as response:
        async for chunk in response.iter_bytes(1024):
            stream.write(chunk)
            yield chunk

    stream.stop_stream()
    stream.close()
    p.terminate()

# To run the async generator in Jupyter notebook
async for chunk in stream_audio():
    pass  # or do something with each chunk if needed

In [4]:
from voice.stt import STT
from fastapi import UploadFile
stt = STT(api_key=config["llm"]["groq_api_key"])
import io
# Open the file in binary mode ('rb')


In [ ]:
with open("./sample-0.mp3", "rb") as f:
    file_content = f.read()
    # Create UploadFile with the file content
    upload_file = UploadFile(filename="sample-0.mp3", file=io.BytesIO(file_content))
    result = await stt.generate_audio(upload_file)
result

In [ ]:
import aiohttp
import asyncio
import io

async def test_stt_endpoint():
    # Replace with your actual audio file path
    audio_file_path = "./sample-0.mp3"
    
    # Read the audio file
    with open(audio_file_path, "rb") as f:
        file_content = f.read()
    
    # Create the form data
    data = aiohttp.FormData()
    data.add_field('input',
                   io.BytesIO(file_content),
                   filename='audio.mp3',
                   content_type='audio/mpeg')
    
    # Make the request
    async with aiohttp.ClientSession() as session:
        async with session.post('http://localhost:8000/stt', data=data) as response:
            # Print status
            print(f"Status: {response.status}")
            
            # Get the response
            result = await response.text()
            print(f"Response: {result}")

# Run the test
await test_stt_endpoint()

In [ ]:
import asyncio
import aiohttp
import uuid
import os
from datetime import datetime

async def test_chat_and_save_audio():
    API_URL = "http://localhost:8000/chat-to-audio"
    
    request_data = {
        "conversation_id": str(uuid.uuid4()),
        "message": {
            "id": str(uuid.uuid4()),
            "role": "user",
            "content": "Bạn có nói tục được không ?"
        }
    }
    
    # Tạo thư mục audio nếu chưa tồn tại
    if not os.path.exists('audio'):
        os.makedirs('audio')
    
    # Tạo tên file với timestamp
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"audio/response_{timestamp}.mp3"
    
    async with aiohttp.ClientSession() as session:
        async with session.post(API_URL, json=request_data) as response:
            print(f"Status: {response.status}")
            
            if response.status == 200:
                # Mở file để ghi
                with open(filename, 'wb') as f:
                    while True:
                        chunk = await response.content.read(1024)
                        if not chunk:
                            break
                        f.write(chunk)
                print(f"Audio đã được lưu tại: {filename}")
            else:
                error_text = await response.text()
                print(f"Error Response: {error_text}")

# For Jupyter notebook
import nest_asyncio
nest_asyncio.apply()

# Run the test
await test_chat_and_save_audio()

In [ ]:
import os
from datetime import datetime
async def test_audio_to_audio():
    API_URL = "http://145.223.21.25:8001/audio-to-audio"
    AUDIO_PATH = "./audio/response_20241113_174510.mp3"
    
    if not os.path.exists(AUDIO_PATH):
        print(f"File không tồn tại: {AUDIO_PATH}")
        return

    # Create output directory if it doesn't exist
    output_dir = "./output_audio"
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Generate output filename with timestamp
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    output_filename = f"{output_dir}/response_{timestamp}.mp3"

    # Tạo form data
    form = aiohttp.FormData()
    form.add_field('conversation_id', str(uuid.uuid4()))
    form.add_field('audio_file', 
                  open(AUDIO_PATH, 'rb'),
                  filename='audio.mp3',
                  content_type='audio/mpeg')

    async with aiohttp.ClientSession() as session:
        async with session.post(API_URL, data=form) as response:
            print(f"Status: {response.status}")
            
            if response.status == 200:
                # Save the audio response to file
                with open(output_filename, 'wb') as f:
                    async for chunk in response.content.iter_chunked(1024):
                        f.write(chunk)
                
                print(f"Đã lưu audio response vào file: {output_filename}")
            else:
                error_text = await response.text()
                print(f"Error Response: {error_text}")

# For Jupyter notebook
import nest_asyncio
nest_asyncio.apply()

# Run the test
await test_audio_to_audio()

In [ ]:
import base64

# Read your audio file
with open('./audio/response.wav', 'rb') as audio_file:
    audio_bytes = audio_file.read()
    base64_audio = base64.b64encode(audio_bytes).decode('utf-8')
    print(base64_audio)  # Copy this output to use in Postman